# Алгоритм Томпсона на батчах (без контекста)

1. На первом батче распределяем юзеров 50% на 50%.
2. Вероятность конверсии каждого варианта распределена по Beta распределению с $\alpha=1$,
$$\beta=1$$.
3. В конце каждого батча пересчитываем вероятности превосходства по точной формуле,
взятой отсюда https://www.johndcook.com//UTMDABTR-005-05.pdf
4. Распределяем трафик в пропорции вероятностей превосходства для каждого варианта
5. Останавливаем эксперимент при достижении определенной вероятности превосходства,
но не раньше определенного дня, чтобы учесть календарные факторы

Потенциальные проблемы:
- слишком рано отдаем трафик победителю
- из-за дисбаланса распределения трафика может быть больше успешных конверсий в этом варианте
(можно попробовать применить нормализацию)


***Реализуем алгоритм***

0. **Инициализация** - *BatchThompson(n_arms)*. Аргумент на вход: число вариантов сплита.
Здесь также инициализируются массивы для параметров Бета-распределений и вероятность превосходства = 0.5.

1. **Метод сплита** - *split_data()*. Исходя из вероятности превосходства вычисляем сплит по вариантам.
Возвращаем данные по конверсии на текущем батче для пересчета Бета-распределений.

2. **Метод изменения параметров распределения** - *.update_beta_params(data)*. Аргументы на вход: numpy массив со значениями конверсии по
каждому варианту. В случае неравномерного распределения по вариантам ставятся пропуски.
 - Проверяем, чтобы число столбцов совпадало с числом вариантов из инициализации.
 - Суммируем нули и единицы и обновляем параметры

3. **Метод пересчета** - *update_prob_super()*. Аргументов нет, так как учитывает измененные параметры
$\alpha$ (накопленное число успешных конверсий) и
$\beta$ (накопленное число неудачных конверсий) для всех вариантов.
 - Считаем по точной формуле
 - Выдаем массив из вероятностей превосходства

4. **Вероятность превосходства** - *prob_super_tuple()*. Аргументов нет, так как берем пересчитанные параметры.
5. **Критерий остановки** (*stopping_criterion*) - условия цикла while. Либо вероятность превосходства выше заданной
величины, либо закончились наблюдения.

In [146]:
import os
import numpy as np
from typing import List, Tuple
from scipy.stats import beta
from tqdm.notebook import tqdm
from AB_classic import get_size_zratio
from MAB import calc_prob_between

# Графики
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import warnings
warnings.filterwarnings("ignore")


def create_directory_plot(folder: str, file_name: str):
    directory_plots = 'Plot/Thompson/' + folder + "/"
    try:
        os.mkdir(directory_plots)
    except:
        pass
    beta_distr_plot = PdfPages(directory_plots + file_name + ".pdf")
    return beta_distr_plot


class BatchThompson:
    def __init__(self, n_arms: np.uint8):
        self._n_arms = n_arms
        self._alphas = np.repeat(1.0, n_arms)
        self._bethas = np.repeat(1.0, n_arms)
        self._prob_superiority_tuple = (0.5, 0.5)

    def split_data_historic(self, data_experiment: np.array, cumulative_observs: List, batch_split_obs: List):
        """
        Split data in every batch iteration
        :param data_experiment: historic data
        :param cumulative_observs: list with cumulative observations for every arm
        :param batch_split_obs: how many observation we must extract this iter
        :return:
        """
        n_rows, n_cols = np.max(batch_split_obs), self._n_arms
        data_split = np.empty((n_rows, n_cols))
        data_split[:] = np.nan

        for i in range(self._n_arms):
            observation_slice_end = np.min([len(data_experiment[cumulative_observs[i] : cumulative_observs[i] + batch_split_obs[i],
                                           i]), batch_split_obs[i]]).item()
            data_split[:observation_slice_end, i] = \
                data_experiment[cumulative_observs[i] : cumulative_observs[i] + batch_split_obs[i], i]
        return data_split


    def split_data_random(self, batch_split_obs: np.array, probs: List):
        """

        :param batch_split_obs: size for every arm
        :param probs: probability for conversion rate
        :return:
        """
        data_split = np.empty((np.max(batch_split_obs), self._n_arms))
        data_split[:] = np.nan
        for i in range(self._n_arms):
            data_split[:batch_split_obs[i], i] = np.random.binomial(n=1, p=probs[i], size=batch_split_obs[i])
        return data_split


    def update_beta_params(self, batch_data: np.array, method:str):
        assert data.shape[1] == self._n_arms
        if method == "summation":
            self._alphas += np.nansum(batch_data, axis=0)
            self._bethas += np.sum(batch_data == 0, axis=0)
        elif method == "normalization":
            S_list =  np.nansum(batch_data, axis=0)  # number of successes in within batch
            F_list = np.sum(batch_data == 0, axis=0)
            M = batch_data.shape[0]
            K = self._n_arms

            adding_alphas = (M / K ) * (np.array(S_list) / (np.array(S_list) + np.array(F_list)))
            adding_bethas = (M / K ) * (1 - np.array(S_list) / (np.array(S_list) + np.array(F_list)))

            adding_alphas = np.nan_to_num(adding_alphas)
            adding_bethas = np.nan_to_num(adding_bethas)

            self._alphas += adding_alphas
            self._bethas += adding_bethas
        return self._alphas, self._bethas


    def update_prob_super(self, method_calc) -> Tuple:
        if method_calc == 'integrating':
            prob_superiority =  calc_prob_between(self._alphas, self._bethas)
            self._prob_superiority_tuple = (prob_superiority, 1 - prob_superiority)
            return self._prob_superiority_tuple


    def create_plots(self, beta_distr_plot, p1, p2, seed, cumulative_observs):
        x = np.linspace(0, 1, 100)
        rv1 = beta(self._alphas[0], self._bethas[0])
        rv2 = beta(self._alphas[1], self._bethas[1])
        fix, ax = plt.subplots()
        ax.plot(x, rv1.pdf(x), label='control')
        ax.plot(x, rv2.pdf(x), label='testing')
        leg = ax.legend();
        plt.title(f"Вероятность превосходства в %: "
                  f"{np.round(tuple(map(lambda x: x * 100, self._prob_superiority_tuple)), 1)} \n"
                  f"Всего юзеров в руки: {cumulative_observs} \n"
                  f"Случайный seed: {seed} \n"
                  f"Теоретические конверсии: {(p1, p2)}",
                  fontdict={"size": 5})
        # plt.savefig("Plot/Thompson/Experiment3/GIFs/" + str(cumulative_observs).replace("[", "_").replace("]", "_") + ".png")
        beta_distr_plot.savefig()
        plt.close()
# Experiment params
p1, p2 = 0.4, 0.42
n_obs_every_arm = get_size_zratio(p1, p2, 0.05, 0.2)
print(f"Нужно наблюдений в каждую руку для выявления эффекта в классическом АБ-тесте: "
      f"{n_obs_every_arm}")
n_obs, batch_size_share, n_arms = n_obs_every_arm * 2, 0.001, 2
batch_size = np.uint16(batch_size_share * n_obs)
# Generating data
data = np.empty(shape=(n_obs_every_arm, n_arms))
seed = np.uint16(np.random.random(size=1) * 100).item()
print(f"Random seed = {seed}")
np.random.seed(seed)

data[:, 0] = np.random.binomial(n=1, p=p1, size=n_obs_every_arm)
data[:, 1] = np.random.binomial(n=1, p=p2, size=n_obs_every_arm)

batchT = BatchThompson(n_arms=2)
probability_superiority = batchT.update_prob_super(method_calc="integrating") # recalculate shares
cumulative_observs = np.repeat(0, n_arms)  # how many observations we extract every iter for every arm

# Plots
folder, file_name = "Experiment4", "batch_size= " + str(round(batch_size_share * 100, 3)) + "%"
beta_distr_plot = create_directory_plot(folder, file_name + "_" + str(seed))

for _ in tqdm(range(n_obs // batch_size)):
    batch_split_obs = np.round(np.array(batch_size) * probability_superiority).astype(np.uint16)  # get number of observations every arm
    # batch_data = batchT.split_data_historic(data_experiment=data, cumulative_observs=cumulative_observs,
    #                                         batch_split_obs=batch_split_obs) # based on earlier generated distr
    batch_data = batchT.split_data_random(batch_split_obs, probs=[p1, p2])  # based on generate batch online

    alphas_batch, bethas_batch = batchT.update_beta_params(batch_data,
                                                           method="normalization" # "normalization", "summation"
                                                           )  # update beta distributions
    cumulative_observs += batch_split_obs  # cumulative sum of observations for every arm
    probability_superiority = batchT.update_prob_super(method_calc="integrating") # recalculate shares
    print(f"Вероятность превосходства на каждом шаге в %: "
          f"{np.round([i * 100 for i in probability_superiority], 1)}")
    batchT.create_plots(beta_distr_plot, p1, p2, seed, cumulative_observs)

    stopping_criterion = (np.max(probability_superiority) >= 0.99) | \
                         (np.max(cumulative_observs) ==  n_obs_every_arm)
    if stopping_criterion:
        break
        print(f"Всего прошло юзеров в каждую руку: {cumulative_observs}")
beta_distr_plot.close()

Нужно наблюдений в каждую руку для выявления эффекта в классическом АБ-тесте: 9489
Random seed = 99


  0%|          | 0/1054 [00:00<?, ?it/s]

Вероятность превосходства на каждом шаге в %: [61.9 38.1]
Вероятность превосходства на каждом шаге в %: [32.2 67.8]
Вероятность превосходства на каждом шаге в %: [34.6 65.4]
Вероятность превосходства на каждом шаге в %: [65.8 34.2]
Вероятность превосходства на каждом шаге в %: [38.2 61.8]
Вероятность превосходства на каждом шаге в %: [35.1 64.9]
Вероятность превосходства на каждом шаге в %: [24.4 75.6]
Вероятность превосходства на каждом шаге в %: [51.5 48.5]
Вероятность превосходства на каждом шаге в %: [51.4 48.6]
Вероятность превосходства на каждом шаге в %: [36.5 63.5]
Вероятность превосходства на каждом шаге в %: [50.8 49.2]
Вероятность превосходства на каждом шаге в %: [40.4 59.6]
Вероятность превосходства на каждом шаге в %: [34.8 65.2]
Вероятность превосходства на каждом шаге в %: [35.4 64.6]
Вероятность превосходства на каждом шаге в %: [38.8 61.2]
Вероятность превосходства на каждом шаге в %: [23.7 76.3]
Вероятность превосходства на каждом шаге в %: [20.2 79.8]
Вероятность пр

In [111]:
batch_size_share * n_obs

306.0

Пытаюсь нарисовать хорошие гифки

In [77]:
import imageio
import glob
images = []
filenames = glob.glob("/home/igor/Appbooster/ABTesting/MAB/Plot/Thompson/Experiment3/GIFs/*.png")
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('/home/igor/Appbooster/ABTesting/MAB/Plot/Thompson/Experiment3/GIFs/' + "results.gif", images)

In [79]:
with imageio.get_writer('/home/igor/Appbooster/ABTesting/MAB/Plot/Thompson/Experiment3/GIFs/' + "results2.gif",
                        mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)